In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [4]:
df_spamCollection = pd.read_csv('C:\DS\Datasets\SpamCollection\SpamCollection\SpamCollection', sep = '\t', names = ['response','message'])

In [6]:
print(df_spamCollection.head(10))
df_spamCollection.shape

  response                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
5     spam  FreeMsg Hey there darling it's been 3 week's n...
6      ham  Even my brother is not like to speak with me. ...
7      ham  As per your request 'Melle Melle (Oru Minnamin...
8     spam  WINNER!! As a valued network customer you have...
9     spam  Had your mobile 11 months or more? U R entitle...


(5572, 2)

In [7]:
df_spamCollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [8]:
df_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [9]:
df_spamCollection['length'] = df_spamCollection['message'].apply(len)

In [11]:
df_spamCollection.head(10)

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147
6,ham,Even my brother is not like to speak with me. ...,77
7,ham,As per your request 'Melle Melle (Oru Minnamin...,160
8,spam,WINNER!! As a valued network customer you have...,157
9,spam,Had your mobile 11 months or more? U R entitle...,154


In [12]:
# Function to get rid of Stopwords and Punctuation
def message_text_process(mess):
    # Get rid of punctuation
    no_punc = [char for char in mess if char not in string.punctuation]
    # Form a sentence in mess
    no_punc = ''.join(no_punc)
    # Eliminate Stopwords
    return [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]

In [13]:
# Verify working of the function
df_spamCollection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [14]:
df_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [16]:
from sklearn.feature_extraction.text import CountVectorizer


In [17]:
bag_of_words_trans = CountVectorizer(analyzer = message_text_process).fit(df_spamCollection['message'])

In [18]:
print(len(bag_of_words_trans.vocabulary_))

11425


In [19]:
mess_bag_of_words = bag_of_words_trans.transform(df_spamCollection['message'])

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_trans = TfidfTransformer().fit(mess_bag_of_words)

In [40]:
mess_tfidf = tfidf_trans.transform(mess_bag_of_words)
print(mess_tfidf)

  (0, 11163)	0.23026685592418913
  (0, 10965)	0.19073428545061483
  (0, 8917)	0.24704652376837993
  (0, 8336)	0.17046869292195632
  (0, 7668)	0.26403384065473806
  (0, 7555)	0.31253856260694546
  (0, 6937)	0.1834692413608692
  (0, 6906)	0.15158474664662352
  (0, 6217)	0.18915557732842803
  (0, 5769)	0.24984711892976424
  (0, 5218)	0.26870593862526665
  (0, 5217)	0.29835184088197164
  (0, 4653)	0.31253856260694546
  (0, 2060)	0.24203960256420656
  (0, 1483)	0.31253856260694546
  (0, 1110)	0.2882862016308418
  (1, 11072)	0.40061560982443056
  (1, 10698)	0.2063637481323008
  (1, 8590)	0.5043405901305854
  (1, 7701)	0.3767401070812794
  (1, 3064)	0.2911995411244838
  (1, 2451)	0.561988811929381
  (2, 11123)	0.19104387220509106
  (2, 11084)	0.15898145347176754
  (2, 10686)	0.13995540820792943
  :	:
  (5568, 6882)	0.31367469776242124
  (5568, 6691)	0.47781076401785183
  (5568, 6354)	0.5575721048646767
  (5568, 4880)	0.3853122086093004
  (5569, 10199)	0.520467167163554
  (5569, 8252)	0.432829

In [35]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(mess_tfidf, df_spamCollection['response'])

In [37]:
message = df_spamCollection['message'][2]
bow_message = bag_of_words_trans.transform([message])
tfidf = tfidf_trans.transform(bow_message)
#print('TFIDF',tfidf)
print(message)
print('Predicted:',spam_detect_model.predict(tfidf)[0])
print('Expected:', df_spamCollection.response[2])

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
Predicted: spam
Expected: spam


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_spamCollection['message'], df_spamCollection['response'], random_state = 1)

In [39]:
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

710     4mths half price Orange line rental & latest c...
3740                           Did you stitch his trouser
2711    Hope you enjoyed your new content. text stop t...
3155    Not heard from U4 a while. Call 4 rude chat pr...
3748    Ü neva tell me how i noe... I'm not at home in...
2389    wiskey Brandy Rum Gin Beer Vodka Scotch Shampa...
3464    i am seeking a lady in the street and a freak ...
772     Lol! U drunkard! Just doing my hair at d momen...
3667    I'm turning off my phone. My moms telling ever...
4955    U coming back 4 dinner rite? Dad ask me so i r...
854     AH POOR BABY!HOPE URFEELING BETTERSN LUV! PROB...
4079                  Gam gone after outstanding innings.
2837                         Nice.nice.how is it working?
1392                  Haha just kidding, papa needs drugs
5533    Hey chief, can you give me a bell when you get...
874     Ugh its been a long day. I'm exhausted. Just w...
4408    Awesome, plan to get here any time after like ...
3990    Ok lor

In [41]:
message_bag_of_words_2 = bag_of_words_trans.transform(X_train)
tfidf_trans = TfidfTransformer().fit(message_bag_of_words_2)

In [42]:
mess_tfidf = tfidf_trans.transform(message_bag_of_words_2)


In [43]:
print(mess_tfidf)

  (0, 10763)	0.24260303218913107
  (0, 9335)	0.25029764451686926
  (0, 9038)	0.22862146381781998
  (0, 8835)	0.2363160761455582
  (0, 8831)	0.16761152422330663
  (0, 8624)	0.2741992935184279
  (0, 7810)	0.2004038403815073
  (0, 7716)	0.21701896217885658
  (0, 7004)	0.20835293940293603
  (0, 6699)	0.17177111385268182
  (0, 5304)	0.22430548938306596
  (0, 3850)	0.21541482222642397
  (0, 3091)	0.2187013831875724
  (0, 2855)	0.2741992935184279
  (0, 1865)	0.17405557753396741
  (0, 1417)	0.15749607315268113
  (0, 669)	0.2741992935184279
  (0, 605)	0.13811556666532693
  (0, 325)	0.25029764451686926
  (0, 55)	0.2741992935184279
  (1, 10646)	0.7071067811865475
  (1, 10096)	0.7071067811865475
  (2, 10759)	0.2789554812857924
  (2, 10554)	0.3599857181901648
  (2, 10378)	0.19792524438142006
  :	:
  (4177, 9676)	0.1955383631834546
  (4177, 9248)	0.25986956683104656
  (4177, 8789)	0.23903329868618378
  (4177, 8283)	0.1649689322278887
  (4177, 8092)	0.16192994446509554
  (4177, 6613)	0.25986956683104

In [44]:
spam_detect_model = MultinomialNB().fit(mess_tfidf, Y_train)

In [48]:
message = X_test
bow_message = bag_of_words_trans.transform([message])
tfidf = tfidf_trans.transform(bow_message)
#print('TFIDF',tfidf)
print(message)
print('Predicted:',spam_detect_model.predict(tfidf)[0])
print('Expected:', Y_test)

1078                         Yep, by the pretty sculpture
4028        Yes, princess. Are you going to make me moan?
958                            Welp apparently he retired
4642                                              Havent.
4674    I forgot 2 ask ü all smth.. There's a card on ...
5461    Ok i thk i got it. Then u wan me 2 come now or...
4210    I want kfc its Tuesday. Only buy 2 meals ONLY ...
4216                           No dear i was sleeping :-P
1603                            Ok pa. Nothing problem:-)
1504                      Ill be there on  &lt;#&gt;  ok.
1783    My uncles in Atlanta. Wish you guys a great se...
3465                                             My phone
5534                         Ok which your another number
4267    The greatest test of courage on earth is to be...
2498    Dai what this da.. Can i send my resume to thi...
4259                        I am late. I will be there at
147     FreeMsg Why haven't you replied to my text? I'...
141           